In [ ]:
from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'ROOT'

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("-([0-9][0-9][0-9]|[0-9][0-9][0-9][0-9])e", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                pulsed_array.append(name)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    return pulsed_array, data_x, data_y

#Code to firstly find the peaks of the fits for the pulses

def fit_data(x_data, y_data, time):
    max_points = []
    time_points = []
    gradients = []
    integral_areas = []
    
    for x_pos, y_pos in zip(x_data, y_data):
        mp = []
        tp = []
        gr = []
        ia = []

        if x_pos == [] and y_pos == []:
            max_points.append(0)
            time_points.append(0)
            gradients.append(0)
            integral_areas.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):
                ifit1 = np.polyfit(x, y, 6)
                ifunc = np.poly1d(ifit1)
                ix_new = np.linspace(x[0], x[-1], 200)
                iy_new = ifunc(ix_new)
                
                min_point = min(iy_new)
                min_index = np.where(iy_new == min_point)[0][0]
                min_point_x = ix_new[min_index]
                
                for i in range(len(x)):
                    if list(x)[i] >= min_point_x and list(x)[i-1] < min_point_x:
                        x_index = i
                        break
                    else:
                        continue
                
                fit = np.polyfit(list(x)[:x_index], list(y)[:x_index], 2)
                func = np.poly1d(fit)
                x_new = np.linspace(x[0], x[x_index], 200)
                y_new = func(x_new)
            
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                ax.plot(x, y)
                ax.plot(ix_new, iy_new)
                ax.plot(x_new, y_new)
                ax.set_title('Test')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                
                max_point = ifunc(x[min_index])
                mp.append(max_point)
                
                t_point = func(time)
                tp.append(t_point)
                
                grad = min(np.gradient(y_new, x_new))
                gr.append(grad)
                
                for i in range(len(x_new)):
                    if x_new[i] >= time and x_new[i-1] < time:
                        time_index = i
                        break
                    else:
                        continue
                
                area = sc.integrate.simpson(y_new[:time_index])
                ia.append(area)
            
            avg_mp = sum(mp)/len(mp)
            max_points.append(avg_mp)
            avg_tp = sum(tp)/len(tp)
            time_points.append(avg_tp)
            avg_gr = sum(gr)/len(gr)
            gradients.append(avg_gr)
            avg_ia = sum(ia)/len(ia)
            integral_areas.append(avg_ia)
        
    return max_points, time_points, gradients, integral_areas

def plotting_data(max_points, time_points, gradients, integral_areas, position, time):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(max_points))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(max_points[4:11]))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(time_points))
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(time_points)[4:11])
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(gradients))
    ax.set_title('Minimum Gradient against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(gradients)[4:11])
    ax.set_title('Minimum Gradient against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(integral_areas))
    ax.set_title(f'Integral area against position at time = {time}ns')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(integral_areas)[4:11])
    ax.set_title(f'Integral area against position at time = {time}ns')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
files = ["ndata_70um.root", "ndata_60um.root", "ndata_50um.root", "ndata_40um.root", "ndata_30um.root", "ndata_20um.root", "ndata_10um.root", "ndata_00um.root", "pdata_10um.root", "pdata_20um.root", "pdata_30um.root", "pdata_40um.root", "pdata_50um.root", "pdata_60um.root", "pdata_70um.root"]
x_values, y_values, pulse_names = [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)

max_points, time_points, gradients, integral_areas = fit_data(x_values, y_values, 3) 
plotting_data(max_points, time_points, gradients, integral_areas, [-70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70], 3)

In [ ]:
def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("-([0-9][0-9][0-9]|[0-9][0-9][0-9][0-9])e", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                pulsed_array.append(name)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    return pulsed_array, data_x, data_y

#Code to firstly find the peaks of the fits for the pulses

def fit_data(x_data, y_data, time):
    max_points = []
    time_points = []
    gradients = []
    integral_areas = []
    
    for x_pos, y_pos in zip(x_data, y_data):
        mp = []
        tp = []
        gr = []
        ia = []

        if x_pos == [] and y_pos == []:
            max_points.append(0)
            time_points.append(0)
            gradients.append(0)
            integral_areas.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):
                min_point = min(list(y))
                x_index = list(y).index(min_point)
                
                fit = np.polyfit(list(x)[:x_index], list(y)[:x_index], 2)
                func = np.poly1d(fit)
                x_new = np.linspace(x[0], x[x_index], 200)
                y_new = func(x_new)
            
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                ax.plot(x, y)
                ax.plot(x_new, y_new)
                ax.set_title('Test')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                
                mp.append(min_point)
                
                t_point = func(time)
                tp.append(t_point)
                
                grad = min(np.gradient(y_new, x_new))
                gr.append(grad)
                
                for i in range(len(x_new)):
                    if x_new[i] >= time and x_new[i-1] < time:
                        time_index = i
                        break
                    else:
                        continue
                
                area = sc.integrate.simpson(y_new[:time_index])
                ia.append(area)
            
            avg_mp = sum(mp)/len(mp)
            max_points.append(avg_mp)
            avg_tp = sum(tp)/len(tp)
            time_points.append(avg_tp)
            avg_gr = sum(gr)/len(gr)
            gradients.append(avg_gr)
            avg_ia = sum(ia)/len(ia)
            integral_areas.append(avg_ia)
        
    return max_points, time_points, gradients, integral_areas

def plotting_data(max_points, time_points, gradients, integral_areas, position, time):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(max_points))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(max_points[4:11]))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(time_points))
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(time_points)[4:11])
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(gradients))
    ax.set_title('Minimum Gradient against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(gradients)[4:11])
    ax.set_title('Minimum Gradient against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(integral_areas))
    ax.set_title(f'Integral area against position at time = {time}ns')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(integral_areas)[4:11])
    ax.set_title(f'Integral area against position at time = {time}ns')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
files = ["ndata_70um.root", "ndata_60um.root", "ndata_50um.root", "ndata_40um.root", "ndata_30um.root", "ndata_20um.root", "ndata_10um.root", "ndata_00um.root", "pdata_10um.root", "pdata_20um.root", "pdata_30um.root", "pdata_40um.root", "pdata_50um.root", "pdata_60um.root", "pdata_70um.root"]
x_values, y_values, pulse_names = [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)

max_points, time_points, gradients, integral_areas = fit_data(x_values, y_values, 3) 
plotting_data(max_points, time_points, gradients, integral_areas, [-70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70], 3)

In [ ]:
def pulsedisplay(file, pulse_graph):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    event = detector.Get(pulse_graph)
    
    data_x = event.GetX()
    data_y = event.GetY()

    integrate = sc.integrate.cumulative_trapezoid(data_y, data_x)
    return event, data_x, data_y, integrate

def collect_counts(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("-1e", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                #print(name)
                pulsed_array.append(name)
    return pulsed_array

In [ ]:
e_40 = collect_counts("e_40um.root")
e_20 = collect_counts("e_20um.root")
e_zero = collect_counts("e_0um.root")
e_n20 = collect_counts("e_20num.root")
e_n40 = collect_counts("e_40num.root")
a = [e_n40, e_n20, e_zero, e_20, e_40]
print(a)

filess = ["e_40num.root", "e_20num.root", "e_0um.root", "e_20um.root", "e_40um.root"]
avg_mine = []
for d, f in zip(a, filess):
    minimum = []
    for i in d:
        e, x, y, integrated = pulsedisplay(f, i)
        minimum.append(min(y))
    avg = sum(minimum)/len(minimum)
    avg_mine.append(np.abs(avg))
print(avg_mine)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot([-40,-20,0,20,40], avg_mine)

In [ ]:
z = np.linspace(0, 0.01, 100)
y = -600000*z + 4500
print(y)
print(sum(y)/len(y))

In [ ]:
c1 = TCanvas( 'c1', 'Testing', 200, 10, 700, 900 )
c1.SetFillColor( 18 )
c1.SetGrid()

gr = TGraph( a[2] )
gr.SetLineColor( 2 )
gr.SetLineWidth( 4 )
gr.SetMarkerColor( 4 )
gr.SetMarkerStyle( 21 )
gr.SetTitle( a[2].GetTitle() )
gr.GetXaxis().SetTitle( f'{a[2].GetXaxis().GetTitle()}' )
gr.GetYaxis().SetTitle( f'{a[2].GetYaxis().GetTitle()}' )
gr.Draw( 'ACP' )

c1.Update()
c1.GetFrame().SetFillColor( 21 )
c1.GetFrame().SetBorderSize( 12 )
c1.Modified()
c1.Update()

e1, x1, y1, integrated1 = pulsedisplay("modules2.root", 'pulse_ev1_px268-28')

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(list(x1)[580:650], list(y1)[580:650])

In [ ]:
e, x, y, integrated = pulsedisplay("modules.root", "pulse_ev4_px257-1")

c1 = TCanvas( 'c1', 'Testing', 200, 10, 700, 900 )
c1.SetFillColor( 18 )
c1.SetGrid()

gr = TGraph( e )
gr.SetLineColor( 2 )
gr.SetLineWidth( 4 )
gr.SetMarkerColor( 4 )
gr.SetMarkerStyle( 21 )
gr.SetTitle( e.GetTitle() )
gr.GetXaxis().SetTitle( f'{e.GetXaxis().GetTitle()}' )
gr.GetYaxis().SetTitle( f'{e.GetYaxis().GetTitle()}' )
gr.Draw( 'ACP' )

c1.Update()
c1.GetFrame().SetFillColor( 21 )
c1.GetFrame().SetBorderSize( 12 )
c1.Modified()
c1.Update()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(x, y)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(list(x)[:-1], integrated)
ax.set_title('Integrated Pulse')
ax.set_ylabel('Cumulative Charge (e)')
ax.set_xlabel('Time (ns)')

In [ ]:
e, x, y, integrated = pulsedisplay("modules.root", "current_ev4_px257-1")

c2 = TCanvas( 'c2', 'Testing', 200, 10, 700, 900 )
c2.SetFillColor( 18 )
c2.SetGrid()

gr1 = TGraph( e )
gr1.SetLineColor( 2 )
gr1.SetLineWidth( 4 )
gr1.SetMarkerColor( 4 )
gr1.SetMarkerStyle( 21 )
gr1.SetTitle( e.GetTitle() )
gr1.GetXaxis().SetTitle( f'{e.GetXaxis().GetTitle()}' )
gr1.GetYaxis().SetTitle( f'{e.GetYaxis().GetTitle()}' )
gr1.Draw( 'ACP' )

c2.Update()
c2.GetFrame().SetFillColor( 21 )
c2.GetFrame().SetBorderSize( 12 )
c2.Modified()
c2.Update()

new_x, new_y = [], []
for v, u in zip(x, y):
    new_x.append(v)
    new_y.append(u)

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
#ax.plot(x, np.gradient(y, x))
#print(max(np.gradient(y, x)))
#ax.set_title('Differentiated Pulse')
#ax.set_ylabel('Current (e)')
#ax.set_xlabel('Time (ns)')

y_max = min(y)
i = list(y).index(y_max)
x_max = x[i]
#print(x_max)

fit1 = np.polyfit(new_x[65:i+2], new_y[65:i+2], 3)
func = np.poly1d(fit1)
x_new = np.linspace(new_x[65], new_x[i+2], 200)
y_new = func(x_new)
#print(func)

print(func(1))

#fit = sc.optimize.curve_fit(fit1, new_x[65:i+4], new_y[65:i+4])

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(abs(np.array(x)[:i+65]), -abs(np.array(y)[:i+65]))
ax.plot(x_new, y_new)
ax.set_title('New')
ax.set_ylabel('Current (e)')
ax.set_xlabel('Time (ns)')
print(min(np.gradient(list(y)[:i], list(x)[:i])))

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(x_new, np.gradient(y_new, x_new))
ax.set_title('Differentiated Pulse')
ax.set_ylabel('Current (e)')
ax.set_xlabel('Time (ns)')

print(min(np.gradient(y_new, x_new)))

y_int = []
for val in list(y):
    if val>0:
        continue
    else:
        y_int.append(val)

print(sc.integrate.simpson(y_new))
#print(np.trapz(np.abs(list(y))[:i]))

In [ ]:
myFile = TFile.Open("modules.root", "read")
location = myFile.Get("DetectorHistogrammer")
location1 = location.Get("detector1")
hits = location1.Get("hit_map;1")
print(type(hits))
a = hits.GetBinContent
print(a)

h2 = TCanvas( 'h2', 'Testing', 200, 10, 700, 900 )
h2.SetFillColor( 18 )
h2.SetGrid()

gr = TH2D( hits )
gr.SetLineColor( 2 )
gr.SetLineWidth( 4 )
gr.SetMarkerColor( 4 )
gr.SetMarkerStyle( 21 )
gr.SetTitle( 'a simple graph' )
gr.GetXaxis().SetTitle( 'X title' )
gr.GetYaxis().SetTitle( 'Y title' )
gr.GetZaxis().SetTitle(' Z title' )
gr.Draw( 'ACP' )

h2.Update()
h2.GetFrame().SetFillColor( 21 )
h2.GetFrame().SetBorderSize( 12 )
h2.Modified()
h2.Update()